In [1]:
import numpy as np
import keras
import utils

import matplotlib.pyplot as plt

print("Keras : {}".format(keras.__version__))
%matplotlib inline 

Using TensorFlow backend.


Keras : 2.0.6


## Importer et pré-traitement des données

In [3]:
(x_train, y_train), (x_test, y_test) = utils.load_cifar10(path="cifar10/")

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train/255.
x_test = x_test/255.

print("x_train.shape : {}".format(x_train.shape))
print("x_train.shape : {}".format(x_train.shape))

x_train.shape : (50000, 32, 32, 3)
x_train.shape : (50000, 32, 32, 3)


In [4]:
from keras.utils.np_utils import to_categorical
y_train_enc = to_categorical(y_train, num_classes=10)
y_test_enc = to_categorical(y_test, num_classes=10)

## Construction du réseau de neurones 

In [7]:
# Point de départ
from keras.models import Sequential
# Couches nécessaires
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Activation, GlobalAveragePooling2D

In [11]:
model = Sequential()

model.add(Conv2D(32, 3, input_shape=(32,32,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(64, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))


model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(128, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(128, 3, padding='same'))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(GlobalAveragePooling2D())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
__________

In [14]:
from keras.utils import plot_model
plot_model(model, to_file='models/model_CNN_CIFAR10.png')

## Apprentissage du réseau de neurones

In [15]:
from keras.optimizers import Adam
adam = Adam(lr=0.01)

In [16]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
history = model.fit(x_train, y_train_enc,
              batch_size=32, epochs=50,
              verbose=2,
              validation_split=0.2,
              shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
640s - loss: 1.6869 - acc: 0.3542 - val_loss: 1.9034 - val_acc: 0.3609
Epoch 2/50
638s - loss: 1.2584 - acc: 0.5398 - val_loss: 1.3828 - val_acc: 0.5054
Epoch 3/50
687s - loss: 1.0283 - acc: 0.6332 - val_loss: 1.2004 - val_acc: 0.5947
Epoch 4/50
604s - loss: 0.8654 - acc: 0.6979 - val_loss: 0.8984 - val_acc: 0.6888
Epoch 5/50
622s - loss: 0.7363 - acc: 0.7488 - val_loss: 0.9652 - val_acc: 0.6887
Epoch 6/50
628s - loss: 0.6426 - acc: 0.7798 - val_loss: 0.8567 - val_acc: 0.7123
Epoch 7/50
666s - loss: 0.5794 - acc: 0.8031 - val_loss: 0.7768 - val_acc: 0.7471
Epoch 8/50


KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'],'r')
plt.plot(history.history['val_acc'],'b')
plt.ylabel('Accuracy %')
plt.xlabel('Training iterations (epochs)')
plt.legend(['Training','Testing'], loc='upper left')
plt.show()

## Evaluer son modèle

In [18]:
res = model.evaluate(x_test, y_test_enc)
print("\n[loss, accuracy] : {}".format(res))

10000/10000 [==============================] - 57s    

[loss, accuracy] : [0.8878232860565185, 0.71889999999999998]


In [19]:
model.save("models/model_CNN_CIFAR10.h5")